# SEQ Task Position Heatmap/Bar Plot 

In [6]:
import os
from os.path import isfile, exists
import numpy as np
from glob import glob
import pandas as pd
import nibabel as nb
from scipy import stats
from scipy.stats import f
from scipy.stats import norm
from math import sqrt
#import nipype.pipeline.engine as pe
#import nipype.interfaces.io as nio
#import nipype.interfaces.freesurfer as fs
#import nipype.interfaces.utility as util
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#import xlrd
Z = norm.ppf

def reject_outliers(data, m=2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

In [7]:
#sids = ['783143']

sids = ['783125', '783126', '783127', '783128', '783129', '783130', '783131', '783132', '783133', '783134',
        '783135', '783136', '783137', '783138', '783139', '783140', '783141', '783142', '783144', '783146',
        '783147', '783148', '783149', '783150', '783151', '783152', '783153', '783154', '783155', '783156',
        '783157', '783158', '783159', '783163'] #34 subjects
proj_dir = '/home/data/madlab/data/mri/seqtrd' 
mask_filenames = []
cope_files = []
for SID in sids:
    subjmasks = glob(proj_dir + '/mask/cortical_rois/{0}/anat_masks/_subject_id_{0}/*'.format(SID))
    subjmasks.sort()
    mask_filenames.append(subjmasks)
    subjcopes = glob(proj_dir + '/frstlvl/seq_item_pos/*_pos*/' + SID + '/modelfit/contrasts/cope02_*_pos*.nii.gz')
    subjcopes.sort()
    cope_files.append(subjcopes)    
    if len(cope_files[-1]) == 0 or len(cope_files[-1]) != 17:
        print(SID)

## Cell to double-check the array indexing

In [8]:
#change first index of cope_files to indicate participant index in sids array
for i, curr_mask in enumerate(mask_filenames[0]):
    print(i, mask_filenames[0][i].split('/')[-1][:-7])

(0, 'cerebellum_bimask_warped')
(1, 'l_ant_hpc_warped')
(2, 'l_cerebellum_bimask_warped')
(3, 'lh-angular-gyrus_fs_warped')
(4, 'lh-caudate_warped')
(5, 'lh-motor_fs_warped')
(6, 'lh-pcc_fs_warped')
(7, 'lh-precuneus_fs_warped')
(8, 'lh-superiortemporalctx_warped')
(9, 'lh_hpc_warped')
(10, 'lh_mpfc_warped')
(11, 'limbicthal_bin_trans_warped')
(12, 'r_ant_hpc_warped')
(13, 'r_cerebellum_bimask_warped')
(14, 'rh-angular-gyrus_fs_warped')
(15, 'rh-caudate_warped')
(16, 'rh-motor_fs_warped')
(17, 'rh-pcc_fs_warped')
(18, 'rh-precuneus_fs_warped')
(19, 'rh-superiortemporalctx_warped')
(20, 'rh_hpc_warped')
(21, 'rh_mpfc_warped')
(22, 'seq_bold_r01_dtype_despike_roi')
(23, 'x_bihemi_ant_hpc_warped')
(24, 'x_bihemi_mpfc_warped')
(25, 'x_combined_antHC_mPFC_mask')


In [9]:
for i, curr_cope in enumerate(cope_files[0]):
    print(i, cope_files[0][i].split('/')[-1][7:-7]) 

(0, 'inseq_pos2')
(1, 'inseq_pos3')
(2, 'inseq_pos4')
(3, 'inseq_pos5')
(4, 'inseq_pos6')
(5, 'ot_pos2')
(6, 'ot_pos3')
(7, 'ot_pos4')
(8, 'ot_pos5')
(9, 'repeat_pos3')
(10, 'repeat_pos4')
(11, 'repeat_pos5')
(12, 'repeat_pos6')
(13, 'skip_pos2')
(14, 'skip_pos3')
(15, 'skip_pos4')
(16, 'skip_pos5')


### Use binarized mask to obtain activation in left & right hemisphere for each region of interest

In [10]:
inseq = {'subjid':[],
         'hp_pos2':[], 'hp_pos3':[], 'hp_pos4':[], 'hp_pos5':[], 'hp_pos6':[],  
         'mpfc_pos2':[], 'mpfc_pos3':[], 'mpfc_pos4':[], 'mpfc_pos5':[], 'mpfc_pos6':[],
         'caud_pos2':[], 'caud_pos3':[], 'caud_pos4':[], 'caud_pos5':[], 'caud_pos6':[],
         'suptemp_pos2':[], 'suptemp_pos3':[], 'suptemp_pos4':[], 'suptemp_pos5':[], 'suptemp_pos6':[]}

ot = {'subjid':[],
      'hp_pos2':[], 'hp_pos3':[], 'hp_pos4':[], 'hp_pos5':[],  
      'mpfc_pos2':[], 'mpfc_pos3':[], 'mpfc_pos4':[], 'mpfc_pos5':[],
      'caud_pos2':[], 'caud_pos3':[], 'caud_pos4':[], 'caud_pos5':[],
      'suptemp_pos2':[], 'suptemp_pos3':[], 'suptemp_pos4':[], 'suptemp_pos5':[]}

repeat = {'subjid':[],
          'hp_pos3':[], 'hp_pos4':[], 'hp_pos5':[], 'hp_pos6':[], 
          'mpfc_pos3':[], 'mpfc_pos4':[], 'mpfc_pos5':[], 'mpfc_pos6':[],
          'caud_pos3':[], 'caud_pos4':[], 'caud_pos5':[], 'caud_pos6':[],
          'suptemp_pos3':[], 'suptemp_pos4':[], 'suptemp_pos5':[], 'suptemp_pos6':[]}

skip = {'subjid':[],
        'hp_pos2':[],'hp_pos3':[], 'hp_pos4':[], 'hp_pos5':[], 
        'mpfc_pos2':[], 'mpfc_pos3':[], 'mpfc_pos4':[], 'mpfc_pos5':[],
        'caud_pos2':[], 'caud_pos3':[], 'caud_pos4':[], 'caud_pos5':[],
        'suptemp_pos2':[], 'suptemp_pos3':[], 'suptemp_pos4':[], 'suptemp_pos5':[]}
           
          
for i in range(len(sids)):
    inseq['subjid'].append(sids[i])
    ot['subjid'].append(sids[i])
    repeat['subjid'].append(sids[i])
    skip['subjid'].append(sids[i])
    #import masks
    lh_hp_img = nb.load(mask_filenames[i][2])
    rh_hp_img = nb.load(mask_filenames[i][7])
    lh_mpfc_img = nb.load(mask_filenames[i][3])
    rh_mpfc_img = nb.load(mask_filenames[i][8])
    lh_caud_img = nb.load(mask_filenames[i][0])
    rh_caud_img = nb.load(mask_filenames[i][5])
    lh_suptemp_img = nb.load(mask_filenames[i][1])
    rh_suptemp_img = nb.load(mask_filenames[i][6])
    #import copes
    inseq_pos2_img = nb.load(cope_files[i][0])
    inseq_pos3_img = nb.load(cope_files[i][1])
    inseq_pos4_img = nb.load(cope_files[i][2])
    inseq_pos5_img = nb.load(cope_files[i][3])
    inseq_pos6_img = nb.load(cope_files[i][4])
    ot_pos2_img = nb.load(cope_files[i][5])
    ot_pos3_img = nb.load(cope_files[i][6])
    ot_pos4_img = nb.load(cope_files[i][7])
    ot_pos5_img = nb.load(cope_files[i][8])
    repeat_pos3_img = nb.load(cope_files[i][9])
    repeat_pos4_img = nb.load(cope_files[i][10])
    repeat_pos5_img = nb.load(cope_files[i][11])
    repeat_pos6_img = nb.load(cope_files[i][12]) 
    skip_pos2_img = nb.load(cope_files[i][13])
    skip_pos3_img = nb.load(cope_files[i][14])
    skip_pos4_img = nb.load(cope_files[i][15])
    skip_pos5_img = nb.load(cope_files[i][16]) 

    pos = ['pos2', 'pos3', 'pos4', 'pos5', 'pos6']    
    #flexible loops to populate trial type dictionaries
    for r in ['hp', 'mpfc', 'caud', 'suptemp']:
        for d in ['inseq', 'ot', 'repeat', 'skip']:      
            if d in ['skip', 'ot']: #if a skip or ot trial
                for p in pos[:-1]: #iterate through pos2-5
                    lh_data = eval('{0}_{1}_img.get_data()[lh_{2}_img.get_data() > 0.]'.format(d,p,r))   
                    rh_data = eval('{0}_{1}_img.get_data()[rh_{2}_img.get_data() > 0.]'.format(d,p,r))
                    bihemi_data = np.concatenate((lh_data, rh_data))
                    eval('{0}["{1}_{2}"].append(bihemi_data)'.format(d,r,p))
            elif d == 'repeat':#if a repeat trial
                for p in pos[1:]: #iterate through pos3-6
                    lh_data = eval('{0}_{1}_img.get_data()[lh_{2}_img.get_data() > 0.]'.format(d,p,r))
                    rh_data = eval('{0}_{1}_img.get_data()[rh_{2}_img.get_data() > 0.]'.format(d,p,r))
                    bihemi_data = np.concatenate((lh_data, rh_data))
                    repeat['{0}_{1}'.format(r,p)].append(bihemi_data) 
            else: #if an inseq trial    
                for p in pos: #iterate through pos2-6
                    lh_data = eval('{0}_{1}_img.get_data()[lh_{2}_img.get_data() > 0.]'.format(d,p,r))
                    rh_data = eval('{0}_{1}_img.get_data()[rh_{2}_img.get_data() > 0.]'.format(d,p,r))
                    bihemi_data = np.concatenate((lh_data, rh_data))
                    inseq['{0}_{1}'.format(r,p)].append(bihemi_data)
#convert dicitionaries to dataframes                    
inseq_df = pd.DataFrame(inseq) 
ot_df = pd.DataFrame(ot) 
repeat_df = pd.DataFrame(repeat) 
skip_df = pd.DataFrame(skip) 

In [11]:
## MPFC -- pos 3, 4, and 5
mpfc_inseq_df = inseq_df.iloc[:,11:14] #all rows, only mpfc_pos3-5 in inseq_df 
mpfc_ot_df = ot_df.iloc[:,9:12] #all rows, only mpfc_pos3-5 in ot_df 
mpfc_repeat_df = repeat_df.iloc[:,8:11] #all rows, only mpfc_pos3-5 in repeat_df 
mpfc_skip_df = skip_df.iloc[:,9:12] #all rows, only mpfc_pos3-5 in skip_df 
#mpfc_skip_df #Checker!!

In [12]:
## HPC -- pos 3, 4, and 5
hc_inseq_df = inseq_df.iloc[:,6:9]
hc_ot_df = ot_df.iloc[:,5:8]
hc_repeat_df = repeat_df.iloc[:,4:7]
hc_skip_df = skip_df.iloc[:,5:8]
#hc_skip_df

In [13]:
## CAUDATE -- pos 3, 4, and 5
caud_inseq_df = inseq_df.iloc[:,1:4]
caud_ot_df = ot_df.iloc[:,1:4]
caud_repeat_df = repeat_df.iloc[:,0:3]
caud_skip_df = skip_df.iloc[:,1:4]
#caud_skip_df

In [14]:
## SUPERIOR TEMPORAL -- pos 3, 4, and 5
suptemp_inseq_df = inseq_df.iloc[:,17:20]
suptemp_ot_df = ot_df.iloc[:,14:18]
suptemp_repeat_df = repeat_df.iloc[:,13:16]
suptemp_skip_df = skip_df.iloc[:,14:17]
#suptemp_skip_df

# HC OT vs InSeq

In [15]:
print len(hc_inseq_df['hp_pos3'][0])
hc_inseq_df

3372


,hp_pos3,hp_pos4,hp_pos5
0,"[2.42035, 5.39411, 31.8354, -3.74814, -12.3785...","[11.4401, 8.67926, 24.6381, 6.83388, -0.415691...","[14.3165, 9.29442, 15.7898, -3.0652, 1.30981, ..."
1,"[17.0261, 32.9235, 37.1283, -3.63206, -15.1459...","[29.2492, 14.4189, 22.2893, -7.46931, -11.0384...","[80.8423, -6.44576, 32.8062, 9.25605, 2.95595,..."
2,"[0.445592, -8.84982, 0.267716, 14.9445, 14.653...","[-8.89015, -21.8716, 3.45502, 15.1474, 9.2826,...","[-3.83947, -15.7787, -5.9147, 0.271117, 22.441..."
3,"[-0.71553, -80.8426, -2.44829, 22.5102, -7.000...","[1.7943, -87.9061, -2.34084, 20.7075, -6.7925,...","[7.42088, -97.6901, -2.92021, 19.0996, -12.388..."
4,"[82.3835, 70.1741, 28.0419, 134.075, 17.9774, ...","[76.1112, 54.6877, 37.3837, 81.1391, 29.2427, ...","[54.6173, 31.5822, 30.179, 53.4285, 24.9009, -..."
5,"[9.10887, -2.80868, -13.1615, -10.89, -4.33796...","[-12.1917, 11.0342, -8.95669, -1.80292, 1.0185...","[-23.0147, 19.7797, -0.259382, 0.781524, 11.76..."
6,"[-6.63207, -21.6414, -3.99409, 3.99954, 4.9947...","[-9.80187, -30.684, -0.578864, 4.95121, -4.423...","[-1.8447, -26.5984, 1.05156, 6.97808, 0.217181..."
7,"[-3.23804, 1.5865, -16.9007, -5.16925, -2.7344...","[-2.47482, -2.4575, -15.1491, -2.73242, -3.599...","[-7.41299, -8.0326, -18.9665, -12.3582, -1.922..."
8,"[1.23521, -0.23521, 16.4409, 5.38204, 9.93539,...","[4.32875, 2.3067, 14.3975, 14.3838, 11.7866, -...","[8.15584, 3.60549, 12.1494, 0.907224, 3.5653, ..."
9,"[8.38335, -9.90721, -12.5899, -20.6933, 22.690...","[9.66661, -18.2404, -3.83343, -19.8618, 17.067...","[-4.28743, -26.0655, -0.470212, -14.4175, 25.5..."


In [16]:
list1 = [] 
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range(0,1):
    #This give the correlation value and the p value    
    #DIAGONAL
    
    print hc_inseq_df.iloc[i,0]
    print "!!!!!!"
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_ot_df.iloc[i,0])
    
    list1.append(temp)    
    x = np.array(list1)
    #print x
    df=pd.DataFrame(x)
    
    #inseq_pos4_OT_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_ot_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_ot_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_OT_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_ot_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_OT_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_ot_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_OT_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_ot_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_OT_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_ot_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_ot_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_ot_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

[  2.42034793   5.39410591  31.83542442 ...,   2.23516846   4.97961473
 -38.8895607 ]
!!!!!!


In [17]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_HC_inseq_OT_pos3', 'p_HC_inseq_OT_pos3', 'r_HC_inseq_OT_pos4', 'p_HC_inseq_OT_pos4', 
                    'r_HC_inseq_OT_pos5', 'p_HC_inseq_OT_pos5', 'r_HC_inseq_pos3_OT_pos4', 'p_HC_inseq_pos3_OT_pos4', 
                    'r_HC_inseq_pos3_OT_pos5', 'p_HC_inseq_pos3_OT_pos5', 'r_HC_inseq_pos4_OT_pos5', 
                    'p_HC_inseq_pos4_OT_pos5', 'r_HC_inseq_pos4_OT_pos3', 'p_HC_inseq_pos4_OT_pos3', 
                    'r_HC_inseq_pos5_OT_pos3', 'p_HC_inseq_pos5_OT_pos3', 'r_HC_inseq_pos5_OT_pos4', 
                    'p_HC_inseq_pos5_OT_pos4']
final_df

,r_HC_inseq_OT_pos3,p_HC_inseq_OT_pos3,r_HC_inseq_OT_pos4,p_HC_inseq_OT_pos4,r_HC_inseq_OT_pos5,p_HC_inseq_OT_pos5,r_HC_inseq_pos3_OT_pos4,p_HC_inseq_pos3_OT_pos4,r_HC_inseq_pos3_OT_pos5,p_HC_inseq_pos3_OT_pos5,r_HC_inseq_pos4_OT_pos5,p_HC_inseq_pos4_OT_pos5,r_HC_inseq_pos4_OT_pos3,p_HC_inseq_pos4_OT_pos3,r_HC_inseq_pos5_OT_pos3,p_HC_inseq_pos5_OT_pos3,r_HC_inseq_pos5_OT_pos4,p_HC_inseq_pos5_OT_pos4
0,0.093004,6.290321e-08,0.509812,1.501302e-222,0.227844,5.852254e-41,0.600816,0.0,0.357547,3.080007e-102,0.234396,2.592595e-43,0.228622,3.103032e-41,0.175281,1.134179e-24,0.554745,1.757700e-271


In [18]:
#final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/HC_OT_InSeq_RSA_corr.csv')

# HC Repeat vs InSeq

In [19]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range(35):
    #This give the correlation value and the p value
    
    #DIAGONAL    
    #inseq_pos3_repeat_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_repeat_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_repeat_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_repeat_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_repeat_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_repeat_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_repeat_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_repeat_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_repeat_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_repeat_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_repeat_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_repeat_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_repeat_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_repeat_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_repeat_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

IndexError: single positional indexer is out-of-bounds

In [ ]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_HC_inseq_repeat_pos3', 'p_HC_inseq_repeat_pos3', 'r_HC_inseq_repeat_pos4', 'p_HC_inseq_repeat_pos4', 'r_HC_inseq_repeat_pos5', 'p_HC_inseq_repeat_pos5', 'r_HC_inseq_pos3_repeat_pos4', 'p_HC_inseq_pos3_repeat_pos4', 'r_HC_inseq_pos3_repeat_pos5', 'p_HC_inseq_pos3_repeat_pos5', 'r_HC_inseq_pos4_repeat_pos5', 'p_HC_inseq_pos4_repeat_pos5', 'r_HC_inseq_pos4_repeat_pos3', 'p_HC_inseq_pos4_repeat_pos3', 'r_HC_inseq_pos5_repeat_pos3', 'p_HC_inseq_pos5_repeat_pos3', 'r_HC_inseq_pos5_repeat_pos4', 'p_HC_inseq_pos5_repeat_pos4']
final_df

In [ ]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/HC_repeat_InSeq_RSA_corr.csv')

# HC Skip vs InSeq

In [ ]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range(35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_skip_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_skip_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_skip_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_skip_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_skip_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_skip_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_skip_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_skip_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,0], hc_skip_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_skip_pos5
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_skip_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_skip_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,1], hc_skip_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos3
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_skip_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos4
    temp = stats.pearsonr(hc_inseq_df.iloc[i,2], hc_skip_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [ ]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_HC_inseq_skip_pos3', 'p_HC_inseq_skip_pos3', 'r_HC_inseq_skip_pos4', 'p_HC_inseq_skip_pos4', 'r_HC_inseq_skip_pos5', 'p_HC_inseq_skip_pos5', 'r_HC_inseq_pos3_skip_pos4', 'p_HC_inseq_pos3_skip_pos4', 'r_HC_inseq_pos3_skip_pos5', 'p_HC_inseq_pos3_skip_pos5', 'r_HC_inseq_pos4_skip_pos5', 'p_HC_inseq_pos4_skip_pos5', 'r_HC_inseq_pos4_skip_pos3', 'p_HC_inseq_pos4_skip_pos3', 'r_HC_inseq_pos5_skip_pos3', 'p_HC_inseq_pos5_skip_pos3', 'r_HC_inseq_pos5_skip_pos4', 'p_HC_inseq_pos5_skip_pos4']
final_df

In [ ]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/HC_skip_InSeq_RSA_corr.csv')

# mPFC correlations

# mpfc OT vs InSeq

In [ ]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_OT_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_ot_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_OT_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_ot_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_ot_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_OT_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_ot_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_OT_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_ot_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_OT_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_ot_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_OT_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_ot_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_ot_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_ot_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [ ]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_mpfc_inseq_OT_pos3', 'p_mpfc_inseq_OT_pos3', 'r_mpfc_inseq_OT_pos4', 'p_mpfc_inseq_OT_pos4', 'r_mpfc_inseq_OT_pos5', 'p_mpfc_inseq_OT_pos5', 'r_mpfc_inseq_pos3_OT_pos4', 'p_mpfc_inseq_pos3_OT_pos4', 'r_mpfc_inseq_pos3_OT_pos5', 'p_mpfc_inseq_pos3_OT_pos5', 'r_mpfc_inseq_pos4_OT_pos5', 'p_mpfc_inseq_pos4_OT_pos5', 'r_mpfc_inseq_pos4_OT_pos3', 'p_mpfc_inseq_pos4_OT_pos3', 'r_mpfc_inseq_pos5_OT_pos3', 'p_mpfc_inseq_pos5_OT_pos3', 'r_mpfc_inseq_pos5_OT_pos4', 'p_mpfc_inseq_pos5_OT_pos4']
final_df

In [ ]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/mpfc_OT_InSeq_RSA_corr.csv')

# mpfc repeat vs InSeq

In [ ]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_repeat_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_repeat_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_repeat_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_repeat_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_repeat_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_repeat_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_repeat_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_repeat_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_repeat_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_repeat_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_repeat_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_repeat_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_repeat_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_repeat_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_repeat_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [ ]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_mpfc_inseq_repeat_pos3', 'p_mpfc_inseq_repeat_pos3', 'r_mpfc_inseq_repeat_pos4', 'p_mpfc_inseq_repeat_pos4', 'r_mpfc_inseq_repeat_pos5', 'p_mpfc_inseq_repeat_pos5', 'r_mpfc_inseq_pos3_repeat_pos4', 'p_mpfc_inseq_pos3_repeat_pos4', 'r_mpfc_inseq_pos3_repeat_pos5', 'p_mpfc_inseq_pos3_repeat_pos5', 'r_mpfc_inseq_pos4_repeat_pos5', 'p_mpfc_inseq_pos4_repeat_pos5', 'r_mpfc_inseq_pos4_repeat_pos3', 'p_mpfc_inseq_pos4_repeat_pos3', 'r_mpfc_inseq_pos5_repeat_pos3', 'p_mpfc_inseq_pos5_repeat_pos3', 'r_mpfc_inseq_pos5_repeat_pos4', 'p_mpfc_inseq_pos5_repeat_pos4']
final_df

In [ ]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/mpfc_repeat_InSeq_RSA_corr.csv')

# mpfc Skip vs InSeq

In [ ]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_skip_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_skip_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_skip_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_skip_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_skip_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_skip_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_skip_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_skip_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,0], mpfc_skip_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_skip_pos5
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_skip_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_skip_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,1], mpfc_skip_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos3
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_skip_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos4
    temp = stats.pearsonr(mpfc_inseq_df.iloc[i,2], mpfc_skip_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [ ]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_mpfc_inseq_skip_pos3', 'p_mpfc_inseq_skip_pos3', 'r_mpfc_inseq_skip_pos4', 'p_mpfc_inseq_skip_pos4', 'r_mpfc_inseq_skip_pos5', 'p_mpfc_inseq_skip_pos5', 'r_mpfc_inseq_pos3_skip_pos4', 'p_mpfc_inseq_pos3_skip_pos4', 'r_mpfc_inseq_pos3_skip_pos5', 'p_mpfc_inseq_pos3_skip_pos5', 'r_mpfc_inseq_pos4_skip_pos5', 'p_mpfc_inseq_pos4_skip_pos5', 'r_mpfc_inseq_pos4_skip_pos3', 'p_mpfc_inseq_pos4_skip_pos3', 'r_mpfc_inseq_pos5_skip_pos3', 'p_mpfc_inseq_pos5_skip_pos3', 'r_mpfc_inseq_pos5_skip_pos4', 'p_mpfc_inseq_pos5_skip_pos4']
final_df

In [ ]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/mpfc_skip_InSeq_RSA_corr.csv')

#    CAUDATE 

# caudate OT vs InSeq

In [ ]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation (r) value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_OT_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_ot_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_OT_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_ot_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_ot_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_OT_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_ot_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_OT_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_ot_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_OT_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_ot_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_OT_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_ot_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_ot_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_ot_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [ ]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
#Adding column names
final_df.columns = ['r_caud_inseq_OT_pos3', 'p_caud_inseq_OT_pos3', 'r_caud_inseq_OT_pos4', 'p_caud_inseq_OT_pos4', 'r_caud_inseq_OT_pos5', 'p_caud_inseq_OT_pos5', 'r_caud_inseq_pos3_OT_pos4', 'p_caud_inseq_pos3_OT_pos4', 'r_caud_inseq_pos3_OT_pos5', 'p_caud_inseq_pos3_OT_pos5', 'r_caud_inseq_pos4_OT_pos5', 'p_caud_inseq_pos4_OT_pos5', 'r_caud_inseq_pos4_OT_pos3', 'p_caud_inseq_pos4_OT_pos3', 'r_caud_inseq_pos5_OT_pos3', 'p_caud_inseq_pos5_OT_pos3', 'r_caud_inseq_pos5_OT_pos4', 'p_caud_inseq_pos5_OT_pos4']
final_df

In [ ]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/caud_OT_InSeq_RSA_corr.csv')

# caudate repeat vs InSeq

In [ ]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_repeat_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_repeat_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_repeat_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_repeat_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_repeat_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_repeat_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_repeat_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_repeat_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_repeat_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_repeat_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_repeat_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_repeat_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_repeat_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_repeat_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_repeat_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [ ]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_caud_inseq_repeat_pos3', 'p_caud_inseq_repeat_pos3', 'r_caud_inseq_repeat_pos4', 'p_caud_inseq_repeat_pos4', 'r_caud_inseq_repeat_pos5', 'p_caud_inseq_repeat_pos5', 'r_caud_inseq_pos3_repeat_pos4', 'p_caud_inseq_pos3_repeat_pos4', 'r_caud_inseq_pos3_repeat_pos5', 'p_caud_inseq_pos3_repeat_pos5', 'r_caud_inseq_pos4_repeat_pos5', 'p_caud_inseq_pos4_repeat_pos5', 'r_caud_inseq_pos4_repeat_pos3', 'p_caud_inseq_pos4_repeat_pos3', 'r_caud_inseq_pos5_repeat_pos3', 'p_caud_inseq_pos5_repeat_pos3', 'r_caud_inseq_pos5_repeat_pos4', 'p_caud_inseq_pos5_repeat_pos4']
final_df

In [ ]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/caud_repeat_InSeq_RSA_corr.csv')

# caudate Skip vs InSeq

In [ ]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_skip_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_skip_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_skip_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_skip_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_skip_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_skip_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_skip_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_skip_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,0], caud_skip_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_skip_pos5
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_skip_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_skip_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,1], caud_skip_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos3
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_skip_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos4
    temp = stats.pearsonr(caud_inseq_df.iloc[i,2], caud_skip_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [ ]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_caud_inseq_skip_pos3', 'p_caud_inseq_skip_pos3', 'r_caud_inseq_skip_pos4', 'p_caud_inseq_skip_pos4', 'r_caud_inseq_skip_pos5', 'p_caud_inseq_skip_pos5', 'r_caud_inseq_pos3_skip_pos4', 'p_caud_inseq_pos3_skip_pos4', 'r_caud_inseq_pos3_skip_pos5', 'p_caud_inseq_pos3_skip_pos5', 'r_caud_inseq_pos4_skip_pos5', 'p_caud_inseq_pos4_skip_pos5', 'r_caud_inseq_pos4_skip_pos3', 'p_caud_inseq_pos4_skip_pos3', 'r_caud_inseq_pos5_skip_pos3', 'p_caud_inseq_pos5_skip_pos3', 'r_caud_inseq_pos5_skip_pos4', 'p_caud_inseq_pos5_skip_pos4']
final_df

In [ ]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/caud_skip_InSeq_RSA_corr.csv')

# Superior Temporal Gyrus

### Now we will do a RSA to compare OutSeq and InSeq on pos 3, 4, and 5. We will do this for all the 3 different trials (OT/repeat/skip), and we use the superior temporal gyrus as the mask. 

### RSA of Ordinal Transfers vs. InSeq in the Superior Temporal Gyrus

In [ ]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation (r) value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_OT_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_ot_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_OT_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_ot_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_ot_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_OT_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_ot_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_OT_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_ot_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_OT_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_ot_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_OT_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_ot_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_ot_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_OT_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_ot_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [ ]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
#Adding column names
final_df.columns = ['r_suptemp_inseq_OT_pos3', 'p_suptemp_inseq_OT_pos3', 'r_suptemp_inseq_OT_pos4', 'p_suptemp_inseq_OT_pos4', 'r_suptemp_inseq_OT_pos5', 'p_suptemp_inseq_OT_pos5', 'r_suptemp_inseq_pos3_OT_pos4', 'p_suptemp_inseq_pos3_OT_pos4', 'r_suptemp_inseq_pos3_OT_pos5', 'p_suptemp_inseq_pos3_OT_pos5', 'r_suptemp_inseq_pos4_OT_pos5', 'p_suptemp_inseq_pos4_OT_pos5', 'r_suptemp_inseq_pos4_OT_pos3', 'p_suptemp_inseq_pos4_OT_pos3', 'r_suptemp_inseq_pos5_OT_pos3', 'p_suptemp_inseq_pos5_OT_pos3', 'r_suptemp_inseq_pos5_OT_pos4', 'p_suptemp_inseq_pos5_OT_pos4']
final_df

In [ ]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/suptemp_OT_InSeq_RSA_corr.csv')

### RSA of Repeats vs. InSeq in the Superior Temporal Gyrus

In [ ]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_repeat_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_repeat_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_repeat_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_repeat_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_repeat_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_repeat_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_repeat_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_repeat_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_repeat_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_repeat_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_repeat_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_repeat_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_repeat_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_repeat_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_repeat_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_repeat_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [ ]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_suptemp_inseq_repeat_pos3', 'p_suptemp_inseq_repeat_pos3', 'r_suptemp_inseq_repeat_pos4', 'p_suptemp_inseq_repeat_pos4', 'r_suptemp_inseq_repeat_pos5', 'p_suptemp_inseq_repeat_pos5', 'r_suptemp_inseq_pos3_repeat_pos4', 'p_suptemp_inseq_pos3_repeat_pos4', 'r_suptemp_inseq_pos3_repeat_pos5', 'p_suptemp_inseq_pos3_repeat_pos5', 'r_suptemp_inseq_pos4_repeat_pos5', 'p_suptemp_inseq_pos4_repeat_pos5', 'r_suptemp_inseq_pos4_repeat_pos3', 'p_suptemp_inseq_pos4_repeat_pos3', 'r_suptemp_inseq_pos5_repeat_pos3', 'p_suptemp_inseq_pos5_repeat_pos3', 'r_suptemp_inseq_pos5_repeat_pos4', 'p_suptemp_inseq_pos5_repeat_pos4']
final_df

In [ ]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/suptemp_repeat_InSeq_RSA_corr.csv')

### RSA of Skips vs. InSeq in the Superior Temporal Gyrus

In [ ]:
from scipy import stats

#df = pd.DataFrame() #pandas df
list1 = [] #List
list2 = []
list3 = []
list4 = []
list5 = []
list6 = []
list7 = []
list8 = []
list9 = []

    
for i in range (0,35):
    #This give the correlation value and the p value
    
    #DIAGONAL
    
    #inseq_pos3_skip_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_skip_df.iloc[i,0])
    list1.append(temp)    
    x = np.array(list1)
    df=pd.DataFrame(x)
    
    #inseq_pos4_skip_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_skip_df.iloc[i,1])
    list2.append(temp)
    x = np.array(list2)
    df2=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_skip_df.iloc[i,2])
    list3.append(temp)
    x = np.array(list3)
    df3=pd.DataFrame(x)
    
    
    #UNDER DIAGONAL
    
    #inseq_pos3_skip_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_skip_df.iloc[i,1])
    list4.append(temp)    
    x = np.array(list4)
    df4=pd.DataFrame(x)
    
    #inseq_pos3_skip_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,0], suptemp_skip_df.iloc[i,2])
    list5.append(temp)    
    x = np.array(list5)
    df5=pd.DataFrame(x)
    
    #inseq_pos4_skip_pos5
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_skip_df.iloc[i,2])
    list6.append(temp)    
    x = np.array(list6)
    df6=pd.DataFrame(x)
    
    #ABOVE DIAGONAL
    
    #inseq_pos4_skip_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,1], suptemp_skip_df.iloc[i,0])
    list7.append(temp)    
    x = np.array(list7)
    df7=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos3
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_skip_df.iloc[i,0])
    list8.append(temp)    
    x = np.array(list8)
    df8=pd.DataFrame(x)
    
    #inseq_pos5_skip_pos4
    temp = stats.pearsonr(suptemp_inseq_df.iloc[i,2], suptemp_skip_df.iloc[i,1])
    list9.append(temp)    
    x = np.array(list9)
    df9=pd.DataFrame(x)
    

In [ ]:
final_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9],  axis=1)
final_df.columns = ['r_suptemp_inseq_skip_pos3', 'p_suptemp_inseq_skip_pos3', 'r_suptemp_inseq_skip_pos4', 'p_suptemp_inseq_skip_pos4', 'r_suptemp_inseq_skip_pos5', 'p_suptemp_inseq_skip_pos5', 'r_suptemp_inseq_pos3_skip_pos4', 'p_suptemp_inseq_pos3_skip_pos4', 'r_suptemp_inseq_pos3_skip_pos5', 'p_suptemp_inseq_pos3_skip_pos5', 'r_suptemp_inseq_pos4_skip_pos5', 'p_suptemp_inseq_pos4_skip_pos5', 'r_suptemp_inseq_pos4_skip_pos3', 'p_suptemp_inseq_pos4_skip_pos3', 'r_suptemp_inseq_pos5_skip_pos3', 'p_suptemp_inseq_pos5_skip_pos3', 'r_suptemp_inseq_pos5_skip_pos4', 'p__suptemp_inseq_pos5_skip_pos4']
final_df

In [ ]:
final_df.to_csv(r'/home/data/madlab/scripts/seqtrd/rsa/suptemp_skip_InSeq_RSA_corr.csv')

# The end